In [1]:
from pyomo.environ import AbstractModel, Set, Param, Var, Constraint, ConstraintList, Objective, NonNegativeIntegers, \
    NonNegativeReals, Reals, maximize
import os, re
os.environ['path'] = 'K:\\COIN-OR\\1.8.0\\win32-msvc12\\bin' + os.pathsep + os.environ['path']

In [359]:
target_cals = 2400
target_protein = 300 / target_cals * 4 * 100
#target_macros = {'fat': 30, 'carbs': 25, 'protein': 45, 'fiber': 30}  # fat loss
#target_macros = {'fat': 15, 'carbs': 55, 'protein': 30, 'fiber': 30}  # muscle gain
target_macros = {'fat': 15, 'carbs': 100 - 15 - target_protein,
                 'protein': target_protein, 'fiber': 38}  # muscle gain
#target_macros = {'fat': 62.5, 'carbs': 5, 'protein': 32.5, 'fiber': 0}  # keto
targets = {'fat': target_macros['fat'] / 9 * target_cals / 100,
           'carbs': target_macros['carbs'] / 4 * target_cals / 100,
           'protein': target_macros['protein'] / 4 * target_cals / 100,
           'fiber': target_macros['fiber']}
print('targets =', targets)
amounts = {
    '0% yogurt': 15,
    #'maple syrup': (0,20),
    #'yogurt': 170,
    #'blueberries': 37,  # 1/4 cup
    #'granola': 30,
    #'celery': (0,65),
    #'peanut butter': (0,30),
    #'carrot': (0,65),
    #'hummus': (0,30),
    'chicken': (0,18),
    'smoothie': 1,
    'whey protein': 26,
    'broccoli': (0,156),
    #'cheese stick': (0,3),
    #'cheese': 1,
    'lettuce': 15,
    'tomatoes': 35,
    #'pickle': 4,
    #'ranch dressing': (20,30),
    #'tuna': 1,
    #'mayonnaise': 30,
    'mustard': 20,
    #'dill relish': 30,
    'gummies': (0,4),
    #'nofat cottage cheese': (0,226),
    #'fruit cup': (0,1),
    #'bbq sauce': (20,30),
    'almonds': (0, 56),
    #'mixed nuts': (0, 64),
    #'pecans': (0, 56),
    'egg': (0,3),
    'supergrains': (0,1),
    'apple': (0,2),
    #'bacon': 60,
    #'fish oil': 3,
    #'blue cheese': 28,
    'heritage flakes': 53,
    'skim milk': 244,
    'daddy bar': (0,1),
    #'spicy crackers': (0,6),
    'bread': 2,
    'deli turkey': 56,
    '0% fruit yogurt': (0,3),
    'tuna': (0,2)
    #'spinach': 60,
    #'olive oil': 1,
    #'cheese': 1,
}
ints = ['egg', 'cheese stick', 'cheese brick', 'yogurt/maple syrup', 'smoothie', 'apple', 'banana',
        'cottage cheese', 'gummies', 'fruit yogurt', 'bread', 'pickle', 'supergrains', 'olive oil',
        'daddy bar', 'spicy crackers', '0% fruit yogurt', 'fruit cup', 'tuna']

targets = {'fat': 40.0, 'carbs': 210.0, 'protein': 300.0, 'fiber': 38}


In [360]:
from collections import namedtuple
Food = namedtuple('Food', ['fat', 'carbs', 'protein', 'sodium', 'fiber', 'units'])
food = {
    'soymilk': Food(fat=1.99/243, carbs=10.01/243, protein=6/243, sodium=95/243, fiber=1/243, units='grams'),  # 1 cup = 243 g
    'egg': Food(fat=6.7, carbs=0.98, protein=6.09, sodium=88, fiber=0, units='large'),  # 1 large = 61 g
    '0% yogurt/maple syrup': Food(fat=0.01/170+0.0198, carbs=20.41/170, protein=18.01/170, sodium=67/170, fiber=0, units='grams yogurt'),  # 1 container = 170 g
    '0% yogurt': Food(fat=0.01/170, carbs=0, protein=18.01/170, sodium=67/170, fiber=0, units='grams'),  # 1 container = 170 g
    '5% yogurt': Food(fat=11/227, carbs=7/227, protein=20/227, sodium=80/227, fiber=0, units='grams'),  # 1 container = 170 g
    'blueberries': Food(fat=0.0033, carbs=0.1449, protein=0.0074, sodium=0.0001, fiber=0.0024, units='grams'),  # 1 container = 170 g
    '0% yogurt/blueberries/maple syrup': Food(fat=0.19/170, carbs=28.52/170, protein=18.42/170, sodium=68/170, fiber=1.3/170, units='grams yogurt'),  # 1 container = 170 g
    'celery': Food(fat=0.0017, carbs=0.0297, protein=0.0069, sodium=0.8, fiber=0.016, units='grams'),  # 100 g
    'peanut butter': Food(fat=25/30, carbs=6/30, protein=9/30, sodium=2/30, fiber=2/30, units='grams'),  # 2 tbsp = 32 g
    'carrot': Food(fat=0.0024, carbs=0.0958, protein=0.0093, sodium=0.69, fiber=0.028, units='grams'),  # 100 g
    'hummus': Food(fat=2.5/14, carbs=2/14, protein=1/14, sodium=62.5/14, fiber=0.5/14, units='grams'),  # 2 tbsp = 28 g
    'chicken': Food(fat=1.071, carbs=0, protein=9.306, sodium=22.2, fiber=0, units='ounces'),  # 100 g
    'broccoli': Food(fat=0.64/156, carbs=11.2/156, protein=3.72/156, sodium=64/156, fiber=5.2/156, units='grams'),  # 1/2 cup = 78 g
    'cheese stick': Food(fat=6, carbs=0, protein=7, sodium=190, fiber=0, units='sticks'),  # 1 stick
    'lettuce': Food(fat=0.0015, carbs=0.0287, protein=0.0136, sodium=0.28, fiber=0.013, units='grams'),  # 1 cup = 36 g
    'tomatoes': Food(fat=0.002, carbs=0.0389, protein=0.0088, sodium=0.05, fiber=0.012, units='grams'),  # 1 cup = 149 g
    'bell peppers': Food(fat=0.003, carbs=0.0603, protein=0.0099, sodium=0.04, fiber=0.021, units='grams'),  # 1 cup = 149 g
    'vinaigrette': Food(fat=(1+0)/2, carbs=(0+0.1703)/2, protein=(0+0.0049)/2, sodium=(0.02+0.23)/2, fiber=0, units='grams'),  # 1 tbsp = 16 g
    'ranch dressing': Food(fat=5/30, carbs=4/30, protein=1/30, sodium=310/30, fiber=0, units='grams'),  # 1 tbsp = 16 g
    'protein powder': Food(fat=4/30, carbs=12/30, protein=11/30, sodium=0, fiber=12/30, units='grams'),
    'benefiber': Food(fat=0, carbs=7/7.4, protein=0, sodium=0, fiber=5/7.4, units='grams'),
    'almonds': Food(fat=0.4993, carbs=0.2155, protein=0.2115, sodium=0.01, fiber=0.125, units='grams'),
    'skirt steak': Food(fat=3.8, carbs=0, protein=9, sodium=0.189, fiber=0, units='ounce'),
    'salsa': Food(fat=0, carbs=2/30, protein=0, sodium=200/30, fiber=0, units='grams'),
    'cottage cheese': Food(fat=9, carbs=8, protein=26, sodium=800, fiber=0, units='cups'),
    'nofat cottage cheese': Food(fat=0, carbs=10/226, protein=26/226, sodium=820/226, fiber=0, units='grams'),
    'chicken parm': Food(fat=39, carbs=5.72, protein=70.5, sodium=358, fiber=1, units='serving'),
    'smoothie': Food(fat=0.92, carbs=51.22, protein=3.28, sodium=27, fiber=6.3, units='serving'),
    'tomato caper sauce': Food(fat=1.385, carbs=4.2075, protein=1.05, sodium=206.75, fiber=1.4, units='serving'),
    'salmon': Food(fat=1.8, carbs=0, protein=5.62, sodium=12, fiber=0, units='ounces'),
    'tilapia': Food(fat=0.79, carbs=0, protein=7.84, sodium=17, fiber=0, units='ounces'),
    'fruit cup': Food(fat=0.078, carbs=11.722, protein=0.37, sodium=3.6, fiber=1.28, units='package'),
    'maple syrup': Food(fat=0.0006, carbs=0.6704, protein=0.0004, sodium=0.12, fiber=0, units='grams'),
    'chicken sausage': Food(fat=2, carbs=4, protein=5, sodium=160, fiber=0, units='link'),
    'pork sausage': Food(fat=0.248, carbs=0.0093, protein=0.1539, sodium=7.39, fiber=0, units='grams'), # 1 link = 25 g
    'almond milk': Food(fat=0.36, carbs=0.19, protein=0.19, sodium=23, fiber=0, units='ounces'),
    'banana': Food(fat=0.39, carbs=26.95, protein=1.29, sodium=1, fiber=3.1, units='medium'),
    'apple': Food(fat=0.31, carbs=25.13, protein=0.47, sodium=2, fiber=4.4, units='medium'),
    'breaded chicken': Food(fat=1.68, carbs=5.81, protein=10.09, sodium=118, fiber=0.18, units='ounces'),
    'pork chop': Food(fat=1.24, carbs=0, protein=8.10, sodium=19, fiber=0, units='ounces'),
    'chicken meatballs': Food(fat=2.3, carbs=1.31, protein=4.18, sodium=117, fiber=0.48, units='meatball w/ sauce'),
    'cauliflower': Food(fat=0.0045, carbs=0.0411, protein=0.0184, sodium=0, fiber=0.023, units='grams'),
    'asparagus': Food(fat=0.0022, carbs=0.0411, protein=0.0240, sodium=0, fiber=0.020, units='grams'),
    'green beans': Food(fat=0.0028, carbs=0.0788, protein=0.0189, sodium=0, fiber=0.032, units='grams'),
    'canned green beans': Food(fat=0.0041, carbs=0.0419, protein=0.0105, sodium=0, fiber=0.019, units='grams'),
    'canned corn': Food(fat=0.0122, carbs=0.1434, protein=0.0229, sodium=0, fiber=0.020, units='grams'),
    'creamed corn': Food(fat=0.0042, carbs=0.1813, protein=0.0174, sodium=0, fiber=0.012, units='grams'),
    'gummies': Food(fat=0, carbs=3.5, protein=0, sodium=0, fiber=2, units='gummies'),
    'granola': Food(fat=6/30, carbs=18/30, protein=3/30, sodium=105/30, fiber=2/30, units='grams'),
    'bbq sauce': Food(fat=0, carbs=6/32, protein=0, sodium=240/32, fiber=1/32, units='grams'),
    'walnuts': Food(fat=20/30, carbs=4/30, protein=5/30, sodium=0, fiber=2/30, units='grams'),
    'pecans': Food(fat=0.72, carbs=0.14, protein=0.09, sodium=0, fiber=0.1, units='grams'),
    'fish oil': Food(fat=3/2, carbs=1/2, protein=0.5/2, sodium=0, fiber=0, units='pills'),
    'blue cheese': Food(fat=10/28, carbs=0, protein=5/28, sodium=360/28, fiber=0, units='grams'),
    'bacon': Food(fat=0.35, carbs=0.02, protein=0.34, sodium=17, fiber=0, units='grams'),
    'tuna': Food(fat=1.25, carbs=0, protein=27.5, sodium=225, fiber=0, units='cans'),
    'mayonnaise': Food(fat=10/13, carbs=0, protein=0, sodium=90/13, fiber=0, units='grams'),
    'mustard': Food(fat=0, carbs=0, protein=0, sodium=24, fiber=0, units='grams'),
    'dill relish': Food(fat=0, carbs=0, protein=0, sodium=16, fiber=0, units='grams'),
    'heritage flakes': Food(fat=0.375, carbs=0.775, protein=0.125, sodium=4.25, fiber=0.175, units='grams'),
    'milk': Food(fat=0.0198, carbs=0.048, protein=0.033, sodium=0.47, fiber=0, units='grams'),
    'golean crunch': Food(fat=0.059, carbs=0.714, protein=0.167, sodium=1.86, fiber=0.145, units='grams'),
    'daddy bar': Food(fat=2.5, carbs=26, protein=2, sodium=95, fiber=3, units='bars'),
    'bread': Food(fat=3, carbs=19, protein=5, sodium=180, fiber=3, units='slice'),
    'deli turkey': Food(fat=1/56, carbs=0, protein=13/56, sodium=360/56, fiber=0, units='grams'),
    '0% fruit yogurt': Food(fat=0, carbs=18, protein=12, sodium=50, fiber=1, units='container'),
    '2% fruit yogurt': Food(fat=2.5, carbs=16, protein=12, sodium=65, fiber=1, units='container'),
    'orange': Food(fat=0.21, carbs=17.56, protein=1.27, sodium=1, fiber=3.1, units='each'),
    'mixed nuts': Food(fat=0.5357, carbs=0.2143, protein=0.1786, sodium=0, fiber=0.071, units='grams'),
    'cheese brick': Food(fat=8, carbs=0, protein=5, sodium=135, fiber=0, units='each'),
    'cheese': Food(fat=9, carbs=1, protein=7, sodium=180, fiber=0, units='ounces'),
    'supergrains': Food(fat=4.5, carbs=38, protein=7, sodium=180, fiber=5, units='packets'),
    'spicy crackers': Food(fat=3.5/6, carbs=20/6, protein=3/6, sodium=110/6, fiber=3/6, units='crackers'),
    'pickle': Food(fat=0, carbs=1/3, protein=0, sodium=180/3, fiber=0, units='each'),
    'olive oil': Food(fat=14, carbs=0, protein=0, sodium=0, fiber=0, units='tbsp'),
    'spinach': Food(fat=0.0039, carbs=0.0363, protein=0.0286, sodium=0.79, fiber=0.022, units='grams'),
    'skim milk': Food(fat=0.2/245, carbs=12/245, protein=8.3/245, sodium=103/245, fiber=0, units='grams'),
    'whey protein': Food(fat=0.1/26, carbs=2/26, protein=20/26, sodium=35/26, fiber=0, units='grams'),
}
for key, val in food.items():
    food[key]._replace(carbs=val.carbs - val.fiber)
import prettytable
pt = prettytable.PrettyTable(['Name', 'Fat', 'Carbs', 'Protein', 'Sodium', 'Fiber', 'Units'])
for key, val in sorted(food.items()):
    pt.add_row([key, f'{val.fat:.6g}', f'{val.carbs:.6g}', f'{val.protein:.6g}', f'{val.sodium:.6g}',
                f'{val.fiber:.6g}', val.units])
#print(pt)

In [361]:
model = AbstractModel()

In [362]:
model.F = Set(initialize=list(amounts.keys()))
model.N = Set(initialize=['fat', 'carbs', 'protein', 'sodium', 'fiber'])
model.Food = Set(initialize=list(food.keys()))

In [363]:
def a_init(model, f, n):
    return getattr(food[f], n)
model.a = Param(model.Food, model.N, within=NonNegativeReals, initialize=a_init)

In [364]:
model.x = Var(model.F, within=NonNegativeReals)
model.x_fat = Var(within=Reals)
model.x_carbs = Var(within=Reals)
model.x_protein = Var(within=Reals)

In [365]:
def obj_rule(model):
    return 9 * model.x_fat + 4 * model.x_protein + 4 * model.x_carbs
    #return sum(model.x[f] * model.a[f, 'fiber'] for f in model.F)
model.obj = Objective(rule=obj_rule)
#model.obj = Objective(rule=obj_rule, sense=maximize)

In [366]:
def sodium_rule(model):
    return sum(model.x[f] * model.a[f, 'sodium'] for f in model.F) <= 2000
#model.sodium_constraint = Constraint(rule=sodium_rule)

In [367]:
def fiber_rule(model, f):
    return sum(model.x[f] * model.a[f, 'fiber'] for f in model.F) >= targets['fiber']
model.fiber_constraint = Constraint(model.F, rule=fiber_rule)

In [368]:
def x_fat_lb_rule(model):
    return sum(model.x[f] * model.a[f, 'fat'] for f in model.F) - model.x_fat <= targets['fat']
model.x_fat_lb_constraint = Constraint(rule=x_fat_lb_rule)

In [369]:
def x_fat_ub_rule(model):
    return sum(model.x[f] * model.a[f, 'fat'] for f in model.F) + model.x_fat >= targets['fat']
model.x_fat_ub_constraint = Constraint(rule=x_fat_ub_rule)

In [370]:
def x_carbs_lb_rule(model):
    return sum(model.x[f] * model.a[f, 'carbs'] for f in model.F) - model.x_carbs <= targets['carbs']
model.x_carbs_lb_constraint = Constraint(rule=x_carbs_lb_rule)

In [371]:
def x_carbs_ub_rule(model):
    return sum(model.x[f] * model.a[f, 'carbs'] for f in model.F) + model.x_carbs >= targets['carbs']
model.x_carbs_ub_constraint = Constraint(rule=x_carbs_ub_rule)

In [372]:
def x_protein_lb_rule(model):
    return sum(model.x[f] * model.a[f, 'protein'] for f in model.F) - model.x_protein <= targets['protein']
model.x_protein_lb_constraint = Constraint(rule=x_protein_lb_rule)

In [373]:
def x_protein_ub_rule(model):
    return sum(model.x[f] * model.a[f, 'protein'] for f in model.F) + model.x_protein >= targets['protein']
model.x_protein_ub_constraint = Constraint(rule=x_protein_ub_rule)

In [374]:
#model.tomatoes_constraint = Constraint(rule=lambda model: model.x['tomatoes'] - 75 / 72 * model.x['lettuce'] == 0)
#model.dressing_constraint = Constraint(rule=lambda model: model.x['greek dressing'] - 30 / 72 * model.x['lettuce'] == 0)
#model.protein_powder_lb_constraint = Constraint(rule=lambda model: model.x['protein powder'] - 30 * model.x['smoothie'] >= 0)
#model.protein_powder_ub_constraint = Constraint(rule=lambda model: model.x['protein powder'] - 45 * model.x['smoothie'] <= 0)
#model.fruit_cup_constraint = Constraint(rule=lambda model: model.x['cottage cheese'] - model.x['fruit cup'] == 0)

In [375]:
from pyomo.opt import SolverFactory
opt = SolverFactory('cbc')
instance = model.create_instance()
for f in instance.F:
    if isinstance(amounts[f], tuple):
        instance.x[f].setlb(int(amounts[f][0]))
        instance.x[f].setub(int(amounts[f][1]))
        instance.x[f].value = instance.x[f].lb
    else:
        instance.x[f] = amounts[f]
        instance.x[f].fixed = True
    if food[f].units in ['grams', 'ounces'] or f in ints:
        instance.x[f].domain = NonNegativeIntegers
import prettytable
pt_var = prettytable.PrettyTable(['Food', 'Lower', 'Upper', 'Initial'])
pt_fix = prettytable.PrettyTable(['Food', 'Value'])
for f in instance.F:
    if not instance.x[f].fixed:
        pt_var.add_row([f.title(), instance.x[f].lb, instance.x[f].ub, instance.x[f].value])
    else:
        pt_fix.add_row([f.title(), instance.x[f].value])
print(pt_var)
print(pt_fix)
print(f'Integers: {[f for f in instance.F if instance.x[f].domain is NonNegativeIntegers]}')
results = opt.solve(instance)
print(results)

+-----------------+-------+-------+---------+
|       Food      | Lower | Upper | Initial |
+-----------------+-------+-------+---------+
|     Broccoli    |   0   |  156  |    0    |
|     Almonds     |   0   |   56  |    0    |
| 0% Fruit Yogurt |   0   |   3   |    0    |
|   Supergrains   |   0   |   1   |    0    |
|     Chicken     |   0   |   18  |    0    |
|     Gummies     |   0   |   4   |    0    |
|       Egg       |   0   |   3   |    0    |
|      Apple      |   0   |   2   |    0    |
|    Daddy Bar    |   0   |   1   |    0    |
|       Tuna      |   0   |   2   |    0    |
+-----------------+-------+-------+---------+
+-----------------+-------+
|       Food      | Value |
+-----------------+-------+
|      Bread      |   2   |
|    Skim Milk    |  244  |
|     Smoothie    |   1   |
|     Tomatoes    |   35  |
|   Deli Turkey   |   56  |
|     Lettuce     |   15  |
|    0% Yogurt    |   15  |
|   Whey Protein  |   26  |
| Heritage Flakes |   53  |
|     Mustard     | 

In [376]:
fat = carbs = protein = fiber = sodium = 0
import prettytable
import textwrap
pt = prettytable.PrettyTable(['Food', 'Value', 'Input', 'Fat', 'Carbs', 'Protein', 'Sodium', 'Fiber', 'Cals'])
for f in sorted(instance.F):
    value = instance.x[f].value
    fat += value * instance.a[f, 'fat']
    carbs += value * instance.a[f, 'carbs']
    protein += value * instance.a[f, 'protein']
    fiber += value * instance.a[f, 'fiber']
    sodium += value * instance.a[f, 'sodium']
    units = food[f].units
    if f == 'yogurt/maple syrup':
        units = f'{units}, {20*value//170} g maple syrup'
    pt.add_row([f, '\n'.join(textwrap.wrap(f'{value} {units}', 20)), amounts[f], f"{value * instance.a[f, 'fat']:.6g}",
               f"{value * instance.a[f, 'carbs']:.6g}", f"{value * instance.a[f, 'protein']:.6g}",
               f"{value * instance.a[f, 'sodium']:.6g}", f"{value * instance.a[f, 'fiber']:.6g}",
               f"{9*value*instance.a[f,'fat']+4*value*instance.a[f,'carbs']+4*value*instance.a[f,'protein']:.6g}",
               ])

cals = 9 * fat + 4 * carbs + 4 * protein

ts = pt.get_string(junction_char='|').splitlines()
print('\n'.join([ts[1]]+[re.sub(r'-(-+)-', r':\1:', ts[2])]+ts[3:-1]))
print()
print(f'Total Calories = {int(cals)}, Fat = {fat:4.1f}, Carbs = {carbs:.1f}, Protein = {protein:5.1f}, Sodium = {int(sodium):4d}, Fiber = {fiber:4.1f}<br/>')
print(f'   Target Cals = {target_cals}, Fat = {targets["fat"]:.1f}, Carbs = {targets["carbs"]:.1f}, Protein = {targets["protein"]:.1f}, Sodium < 2000, Fiber = {targets["fiber"]:.1f}<br/>')
print(f'Total Macros:  Fat = {fat*9/cals*100:4.1f}%, Carbs = {carbs*4/cals*100:4.1f}%, Protein = {protein*4/cals*100:4.1f}%<br/>')
print(f'     Targets:  Fat = {target_macros["fat"]:4.1f}%, Carbs = {target_macros["carbs"]:4.1f}%, Protein = {target_macros["protein"]:4.1f}%')

|       Food      |     Value     |  Input   |     Fat     |  Carbs  | Protein |  Sodium |  Fiber  |   Cals  |
|:---------------:|:-------------:|:--------:|:-----------:|:-------:|:-------:|:-------:|:-------:|:-------:|
| 0% fruit yogurt | 1.0 container |  (0, 3)  |      0      |    18   |    12   |    50   |    1    |   120   |
|    0% yogurt    |    15 grams   |    15    | 0.000882353 |    0    | 1.58912 | 5.91176 |    0    | 6.36441 |
|     almonds     |   0.0 grams   | (0, 56)  |      0      |    0    |    0    |    0    |    0    |    0    |
|      apple      |   1.0 medium  |  (0, 2)  |     0.31    |  25.13  |   0.47  |    2    |   4.4   |  105.19 |
|      bread      |    2 slice    |    2     |      6      |    38   |    10   |   360   |    6    |   246   |
|     broccoli    |   95.0 grams  | (0, 156) |   0.389744  | 6.82051 | 2.26538 | 38.9744 | 3.16667 | 39.8513 |
|     chicken     |  18.0 ounces  | (0, 18)  |    19.278   |    0    | 167.508 |  399.6  |    0    | 843.534 |
|

|      Food      |    Value    |  Input   |   Fat   |  Carbs  | Protein  |  Sodium |  Fiber   |   Cals  |
|:--------------:|:-----------:|:--------:|:-------:|:-------:|:--------:|:-------:|:--------:|:-------:|
|   bbq sauce    |  30.0 grams | (20, 30) |    0    |  5.625  |    0     |   225   |  0.9375  |   22.5  |
|    broccoli    |   78 grams  |    78    |   0.32  |   5.6   |   1.86   |    32   |   2.6    |  32.72  |
|     carrot     |   65 grams  |    65    |  0.156  |  6.227  |  0.6045  |  44.85  |   1.82   |  28.73  |
|     celery     |   65 grams  |    65    |  0.1105 |  1.9305 |  0.4485  |    52   |   1.04   | 10.5105 |
|     cheese     |  0.0 sticks |  (0, 3)  |    0    |    0    |    0     |    0    |    0     |    0    |
|    chicken     | 12.0 ounces | (0, 12)  |  12.852 |    0    | 111.672  |  266.4  |    0     | 562.356 |
| cottage cheese |    1 cups   |    1     |   4.5   |    4    |    13    |   400   |    0     |  108.5  |
|    fish oil    |   3 pills   |    3     |   4.5   |   1.5   |   0.75   |    0    |    0     |   49.5  |
|   fruit cup    |  1 package  |    1     |  0.078  |  11.722 |   0.37   |   3.6   |   1.28   |  49.07  |
|    gummies     |  4 gummies  |    4     |    0    |    14   |    0     |    0    |    8     |    56   |
|     hummus     |  22.0 grams | (20, 30) | 3.92857 | 3.14286 | 1.57143  | 98.2143 | 0.785714 | 54.2143 |
|    lettuce     |   70 grams  |    70    |  0.105  |  2.009  |  0.952   |   19.6  |   0.91   |  12.789 |
|  maple syrup   |  23.0 grams | (10, 30) |  0.0138 | 15.4192 |  0.0092  |   2.76  |    0     | 61.8378 |
| peanut butter  |  20.0 grams | (20, 30) | 16.6667 |    4    |    6     | 1.33333 | 1.33333  |   190   |
| ranch dressing |  20.0 grams | (20, 30) | 3.33333 | 2.66667 | 0.666667 | 206.667 |    0     | 43.3333 |
|    tomatoes    |  75.0 grams | (40, 75) |   0.15  |  2.9175 |   0.66   |   3.75  |   0.9    |  15.66  |
|    walnuts     |  0.0 grams  | (0, 30)  |    0    |    0    |    0     |    0    |    0     |    0    |
|     yogurt     |  170 grams  |   170    |    0    | 6.74009 | 17.2247  | 63.6564 |    0     |  95.859 |

Total Calories = 1393, Fat = 46.7, Carbs = 87.5, Protein = 155.8, Sodium = 1419, Fiber = 19.6<br/>
Target Cals = 1400, Fat = 46.7, Carbs = 87.5, Protein = 157.5, Sodium < 2000, Fiber = 30.0<br/>
Total Macros:  Fat = 30.2%, Carbs = 25.1%, Protein = 44.7%<br/>
Targets:  Fat = 30.0%, Carbs = 25.0%, Protein = 45.0%

|      Food      |    Value    |   Input    |   Fat   |  Carbs  | Protein |  Sodium |  Fiber   |   Cals  |
|:--------------:|:-----------:|:----------:|:-------:|:-------:|:-------:|:-------:|:--------:|:-------:|
|   bbq sauce    |  26.0 grams |  (20, 30)  |    0    |  4.875  |    0    |   195   |  0.8125  |   19.5  |
|    broccoli    |   78 grams  |     78     |   0.32  |   5.6   |   1.86  |    32   |   2.6    |  32.72  |
|     carrot     |   65 grams  |     65     |  0.156  |  6.227  |  0.6045 |  44.85  |   1.82   |  28.73  |
|     celery     |   65 grams  |     65     |  0.1105 |  1.9305 |  0.4485 |    52   |   1.04   | 10.5105 |
|    chicken     | 11.0 ounces |  (6, 12)   |  11.781 |    0    | 102.366 |  244.2  |    0     | 515.493 |
|    fish oil    |   3 pills   |     3      |   4.5   |   1.5   |   0.75  |    0    |    0     |   49.5  |
|   fruit cup    |  1 package  |     1      |  0.078  |  11.722 |   0.37  |   3.6   |   1.28   |  49.07  |
|    gummies     |  4 gummies  |     4      |    0    |    14   |    0    |    0    |    8     |    56   |
|     hummus     |  15.0 grams |  (15, 30)  | 2.67857 | 2.14286 | 1.07143 | 66.9643 | 0.535714 | 36.9643 |
|    lettuce     |   70 grams  |     70     |  0.105  |  2.009  |  0.952  |   19.6  |   0.91   |  12.789 |
|  maple syrup   |  13.0 grams |  (10, 20)  |  0.0078 |  8.7152 |  0.0052 |   1.56  |    0     | 34.9518 |
| peanut butter  |  20.0 grams |  (15, 30)  | 16.6667 |    4    |    6    | 1.33333 | 1.33333  |   190   |
| ranch dressing |  21.0 grams |  (20, 30)  |   3.5   |   2.8   |   0.7   |   217   |    0     |   45.5  |
|    tomatoes    |  48.0 grams |  (35, 70)  |  0.096  |  1.8672 |  0.4224 |   2.4   |  0.576   | 10.0224 |
|     yogurt     | 192.0 grams | (140, 280) |    0    | 7.61233 | 19.4537 | 71.8943 |    0     | 108.264 |

Total Calories = 1200, Fat = 40.0, Carbs = 75.0, Protein = 135.0, Sodium =  952, Fiber = 18.9<br/>
   Target Cals = 1200, Fat = 40.0, Carbs = 75.0, Protein = 135.0, Sodium < 2000, Fiber = 30.0<br/>
Total Macros:  Fat = 30.0%, Carbs = 25.0%, Protein = 45.0%<br/>
     Targets:  Fat = 30.0%, Carbs = 25.0%, Protein = 45.0%

|      Food      |    Value    | Input |  Fat   | Carbs  | Protein | Sodium | Fiber |   Cals  |
|:--------------:|:-----------:|:-----:|:------:|:------:|:-------:|:------:|:-----:|:-------:|
|     bread      |   2 slice   |   2   |   6    |   38   |    10   |  360   |   6   |   246   |
|   daddy bar    |    1 bars   |   1   |  2.5   |   26   |    2    |   95   |   3   |  134.5  |
|  deli turkey   |   56 grams  |   56  |   1    |   0    |    13   |  360   |   0   |    61   |
|    fish oil    |   3 pills   |   3   |  4.5   |  1.5   |   0.75  |   0    |   0   |   49.5  |
|  fruit yogurt  | 3 container |   3   |   0    |   54   |    36   |  150   |   3   |   360   |
| golean crunch  |   53 grams  |   53  | 3.127  | 37.842 |  8.851  | 98.58  | 7.685 | 214.915 |
|    lettuce     |   35 grams  |   35  | 0.0525 | 1.0045 |  0.476  |  9.8   | 0.455 |  6.3945 |
|      milk      |  244 grams  |  244  | 4.8312 | 11.712 |  8.052  | 114.68 |   0   | 122.537 |
|    mustard     |   15 grams  |   15  |   0    |   0    |    0    |  360   |   0   |    0    |
| protein powder |   30 grams  |   30  |   4    |   12   |    11   |   0    |   12  |   128   |
|    smoothie    |  1 serving  |   1   |  0.92  | 51.22  |   3.28  |   27   |  6.3  |  226.28 |
|    tomatoes    |   20 grams  |   20  |  0.04  | 0.778  |  0.176  |   1    |  0.24 |  4.176  |

Total Calories = 1553, Fat = 27.0, Carbs = 234.1, Protein =  93.6, Sodium = 1576, Fiber = 38.7<br/>
   Target Cals = 1600, Fat = 26.7, Carbs = 220.0, Protein = 120.0, Sodium < 2000, Fiber = 30.0<br/>
Total Macros:  Fat = 15.6%, Carbs = 60.3%, Protein = 24.1%<br/>
     Targets:  Fat = 15.0%, Carbs = 55.0%, Protein = 30.0%

|         Food         |    Value    |  Input   |    Fat     |  Carbs  | Protein |  Sodium |  Fiber  |   Cals  |
|:--------------------:|:-----------:|:--------:|:----------:|:-------:|:-------:|:-------:|:-------:|:-------:|
|      0% yogurt       | 134.0 grams | (0, 170) | 0.00788235 |    0    | 14.1961 | 52.8118 |    0    | 56.8554 |
|        apple         |  1.0 medium |  (0, 1)  |    0.31    |  25.13  |   0.47  |    2    |   4.4   |  105.19 |
|      bbq sauce       |  27.0 grams | (20, 30) |     0      |  5.0625 |    0    |  202.5  | 0.84375 |  20.25  |
|       broccoli       |   78 grams  |    78    |    0.32    |   5.6   |   1.86  |    32   |   2.6   |  32.72  |
|        cheese        |   1 ounces  |    1     |     9      |    1    |    7    |   180   |    0    |   113   |
|       chicken        |  7.0 ounces | (6, 12)  |   7.497    |    0    |  65.142 |  155.4  |    0    | 328.041 |
|         egg          |   3 large   |    3     |    20.1    |   2.94  |  18.27  |   264   |    0    |  265.74 |
|      fruit cup       |  1 package  |    1     |   0.078    |  11.722 |   0.37  |   3.6   |   1.28  |  49.07  |
|     maple syrup      |  17.0 grams | (0, 20)  |   0.0102   | 11.3968 |  0.0068 |   2.04  |    0    | 45.7062 |
| nofat cottage cheese |    1 cups   |    1     |     0      |    10   |    26   |   820   |    0    |   144   |
|      olive oil       |    1 tbsp   |    1     |     14     |    0    |    0    |    0    |    0    |   126   |
|       spinach        |   60 grams  |    60    |   0.234    |  2.178  |  1.716  |   47.4  |   1.32  |  17.682 |

Total Calories = 1304, Fat = 51.6, Carbs = 75.0, Protein = 135.0, Sodium = 1761, Fiber = 10.4<br/>
   Target Cals = 1200, Fat = 40.0, Carbs = 75.0, Protein = 135.0, Sodium < 2000, Fiber = 30.0<br/>
Total Macros:  Fat = 35.6%, Carbs = 23.0%, Protein = 41.4%<br/>
     Targets:  Fat = 30.0%, Carbs = 25.0%, Protein = 45.0%
